In [27]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [39]:
from pyspark.sql.functions import spark_partition_id
data1 = [{'Name':'Jhon','ID':21.528,'Add':'USA'},{'Name':'Joe','ID':3.69,'Add':'USA'},{'Name':'Tina','ID':2.48,'Add':'IND'},{'Name':'Jhon','ID':22.22, 'Add':'USA'},{'Name':'Joe','ID':5.33,'Add':'INA'}]
a = sc.parallelize(data1)
b = spark.createDataFrame(a)
b=b.withColumn("partitionID", spark_partition_id())
b.show()

+---+------+----+-----------+
|Add|    ID|Name|partitionID|
+---+------+----+-----------+
|USA|21.528|Jhon|          0|
|USA|  3.69| Joe|          1|
|IND|  2.48|Tina|          2|
|USA| 22.22|Jhon|          3|
|INA|  5.33| Joe|          3|
+---+------+----+-----------+



In [40]:
data2 = [{'Name':'Atin','ID':21.528,'Add':'India'},{'Name':'Joe','ID':3.69,'Add':'USeA'},{'Name':'Tina','ID':2.48,'Add':'IND'},{'Name':'Jhon','ID':22.22, 'Add':'USdA'},{'Name':'Joe','ID':5.33,'Add':'rsa'}]
c = sc.parallelize(data2)
d = spark.createDataFrame(c)
d=d.withColumn("partitionID", spark_partition_id())
d.show()

+-----+------+----+-----------+
|  Add|    ID|Name|partitionID|
+-----+------+----+-----------+
|India|21.528|Atin|          0|
| USeA|  3.69| Joe|          1|
|  IND|  2.48|Tina|          2|
| USdA| 22.22|Jhon|          3|
|  rsa|  5.33| Joe|          3|
+-----+------+----+-----------+



In [30]:
f = d.join(b,d.Add == b.Add)
f.explain(True)

== Parsed Logical Plan ==
Join Inner, (Add#389 = Add#370)
:- LogicalRDD [Add#389, ID#390, Name#391], false
+- LogicalRDD [Add#370, ID#371, Name#372], false

== Analyzed Logical Plan ==
Add: string, ID: double, Name: string, Add: string, ID: double, Name: string
Join Inner, (Add#389 = Add#370)
:- LogicalRDD [Add#389, ID#390, Name#391], false
+- LogicalRDD [Add#370, ID#371, Name#372], false

== Optimized Logical Plan ==
Join Inner, (Add#389 = Add#370)
:- Filter isnotnull(Add#389)
:  +- LogicalRDD [Add#389, ID#390, Name#391], false
+- Filter isnotnull(Add#370)
   +- LogicalRDD [Add#370, ID#371, Name#372], false

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- SortMergeJoin [Add#389], [Add#370], Inner
   :- Sort [Add#389 ASC NULLS FIRST], false, 0
   :  +- Exchange hashpartitioning(Add#389, 200), ENSURE_REQUIREMENTS, [id=#733]
   :     +- Filter isnotnull(Add#389)
   :        +- Scan ExistingRDD[Add#389,ID#390,Name#391]
   +- Sort [Add#370 ASC NULLS FIRST], false, 0
      +- Exc

In [31]:
f.show()

+---+----+----+---+----+----+
|Add|  ID|Name|Add|  ID|Name|
+---+----+----+---+----+----+
|IND|2.48|Tina|IND|2.48|Tina|
+---+----+----+---+----+----+



In [32]:
from pyspark.sql.functions import broadcast
f = d.join(broadcast(b),d.Add == b.Add)
f.explain(True)

== Parsed Logical Plan ==
Join Inner, (Add#389 = Add#370)
:- LogicalRDD [Add#389, ID#390, Name#391], false
+- ResolvedHint (strategy=broadcast)
   +- LogicalRDD [Add#370, ID#371, Name#372], false

== Analyzed Logical Plan ==
Add: string, ID: double, Name: string, Add: string, ID: double, Name: string
Join Inner, (Add#389 = Add#370)
:- LogicalRDD [Add#389, ID#390, Name#391], false
+- ResolvedHint (strategy=broadcast)
   +- LogicalRDD [Add#370, ID#371, Name#372], false

== Optimized Logical Plan ==
Join Inner, (Add#389 = Add#370), rightHint=(strategy=broadcast)
:- Filter isnotnull(Add#389)
:  +- LogicalRDD [Add#389, ID#390, Name#391], false
+- Filter isnotnull(Add#370)
   +- LogicalRDD [Add#370, ID#371, Name#372], false

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- BroadcastHashJoin [Add#389], [Add#370], Inner, BuildRight, false
   :- Filter isnotnull(Add#389)
   :  +- Scan ExistingRDD[Add#389,ID#390,Name#391]
   +- BroadcastExchange HashedRelationBroadcastMode(List(input[0

In [35]:
f.show()

+---+----+----+---+----+----+
|Add|  ID|Name|Add|  ID|Name|
+---+----+----+---+----+----+
|IND|2.48|Tina|IND|2.48|Tina|
+---+----+----+---+----+----+



In [37]:
f.rdd.getNumPartitions()

4

In [23]:
peopleDF = (
  ("andrea", "medellin"),
  ("rodolfo", "medellin"),
  ("abdul", "bangalore")
)

peopleDF_a = sc.parallelize(peopleDF)
peopleDF_b = spark.createDataFrame(peopleDF_a, ["first_name", "city"])

peopleDF_b.show()

+----------+---------+
|first_name|     city|
+----------+---------+
|    andrea| medellin|
|   rodolfo| medellin|
|     abdul|bangalore|
+----------+---------+



In [24]:
citiesDF = (
  ("medellin", "colombia", 2.5),
  ("bangalore", "india", 12.3)
)




citiesDF_a = sc.parallelize(citiesDF)
citiesDF_b = spark.createDataFrame(citiesDF_a, ("city", "country", "population"))

citiesDF_b.show()

+---------+--------+----------+
|     city| country|population|
+---------+--------+----------+
| medellin|colombia|       2.5|
|bangalore|   india|      12.3|
+---------+--------+----------+



In [25]:
peopleDF_b_j = peopleDF_b.join(
  broadcast(citiesDF_b),
  peopleDF_b.city == citiesDF_b.city
)

peopleDF_b_j.explain(True)

== Parsed Logical Plan ==
Join Inner, (city#319 = city#331)
:- LogicalRDD [first_name#318, city#319], false
+- ResolvedHint (strategy=broadcast)
   +- LogicalRDD [city#331, country#332, population#333], false

== Analyzed Logical Plan ==
first_name: string, city: string, city: string, country: string, population: double
Join Inner, (city#319 = city#331)
:- LogicalRDD [first_name#318, city#319], false
+- ResolvedHint (strategy=broadcast)
   +- LogicalRDD [city#331, country#332, population#333], false

== Optimized Logical Plan ==
Join Inner, (city#319 = city#331), rightHint=(strategy=broadcast)
:- Filter isnotnull(city#319)
:  +- LogicalRDD [first_name#318, city#319], false
+- Filter isnotnull(city#331)
   +- LogicalRDD [city#331, country#332, population#333], false

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- BroadcastHashJoin [city#319], [city#331], Inner, BuildRight, false
   :- Filter isnotnull(city#319)
   :  +- Scan ExistingRDD[first_name#318,city#319]
   +- Broadca

In [26]:
peopleDF_b_j = peopleDF_b.join(
  citiesDF_b,
  peopleDF_b.city == citiesDF_b.city
)

peopleDF_b_j.explain(True)

== Parsed Logical Plan ==
Join Inner, (city#319 = city#331)
:- LogicalRDD [first_name#318, city#319], false
+- LogicalRDD [city#331, country#332, population#333], false

== Analyzed Logical Plan ==
first_name: string, city: string, city: string, country: string, population: double
Join Inner, (city#319 = city#331)
:- LogicalRDD [first_name#318, city#319], false
+- LogicalRDD [city#331, country#332, population#333], false

== Optimized Logical Plan ==
Join Inner, (city#319 = city#331)
:- Filter isnotnull(city#319)
:  +- LogicalRDD [first_name#318, city#319], false
+- Filter isnotnull(city#331)
   +- LogicalRDD [city#331, country#332, population#333], false

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- SortMergeJoin [city#319], [city#331], Inner
   :- Sort [city#319 ASC NULLS FIRST], false, 0
   :  +- Exchange hashpartitioning(city#319, 200), ENSURE_REQUIREMENTS, [id=#683]
   :     +- Filter isnotnull(city#319)
   :        +- Scan ExistingRDD[first_name#318,city#319]
   +- 

#### Notice how the parsed, analyzed, and optimized logical plans all contain ResolvedHint isBroadcastable=true because the broadcast() function was used. This hint isn’t included when the broadcast() function isn’t used.

## Automatic Detection

- In many cases, Spark can automatically detect whether to use a broadcast join or not, depending on the size of the data. If Spark can detect that one of the joined DataFrames is small (10 MB by default), Spark will automatically broadcast it for us. The code below:

In [38]:
bigTable = spark.range(1, 100000000)
smallTable = spark.range(1, 10000) # size estimated by Spark - auto-broadcast
joinedNumbers = smallTable.join(bigTable, "id")
joinedNumbers.explain(True)

== Parsed Logical Plan ==
'Join UsingJoin(Inner,Buffer(id))
:- Range (1, 10000, step=1, splits=Some(4))
+- Range (1, 100000000, step=1, splits=Some(4))

== Analyzed Logical Plan ==
id: bigint
Project [id#276L]
+- Join Inner, (id#276L = id#274L)
   :- Range (1, 10000, step=1, splits=Some(4))
   +- Range (1, 100000000, step=1, splits=Some(4))

== Optimized Logical Plan ==
Project [id#276L]
+- Join Inner, (id#276L = id#274L)
   :- Range (1, 10000, step=1, splits=Some(4))
   +- Range (1, 100000000, step=1, splits=Some(4))

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [id#276L]
   +- BroadcastHashJoin [id#276L], [id#274L], Inner, BuildLeft, false
      :- BroadcastExchange HashedRelationBroadcastMode(List(input[0, bigint, false]),false), [id=#585]
      :  +- Range (1, 10000, step=1, splits=4)
      +- Range (1, 100000000, step=1, splits=4)



- However, in the previous case, Spark did not detect that the small table could be broadcast. 
- The reason is that Spark will not determine the size of a local collection because it might be big, and evaluating its size may be an O(N) operation, which can defeat the purpose before any computation is made.

- Spark will perform auto-detection when
    - it constructs a DataFrame from scratch, e.g. spark.range
    - it reads from files with schema and/or size information, e.g. Parquet

## Configuring Broadcast Join Detection

- The threshold for automatic broadcast join detection can be tuned or disabled.
- The configuration is spark.sql.autoBroadcastJoinThreshold, and the value is taken in bytes. If you want to configure it to another number, we can set it in the SparkSession:

In [41]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", 104857600)

- or deactivate it altogether by setting the value to -1.

In [39]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1) 